In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing Modules


In [4]:
import os
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import spectrogram
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Input, GlobalAveragePooling2D, Multiply, Reshape


In [5]:
# Preprocessing function to load data
def load_patient_data(patient_num, types, num_segments):
    all_X = []
    all_Y = []
    base_path = f'/kaggle/input/seizure-prediction/Patient_{patient_num}/Patient_{patient_num}/'
    
    for i, typ in enumerate(types):
        for j in range(num_segments):
            fl = os.path.join(base_path, '{}_{}.mat'.format(typ, str(j + 1).zfill(4)))
            data = scipy.io.loadmat(fl)
            k = typ.replace(f'Patient_{patient_num}_', '') + '_'
            d_array = data[k + str(j + 1)][0][0][0]
            
            lst = list(range(3000000))  # Adjust for 10 minutes
            for m in lst[::5000]:  # Create a spectrogram every 1 second (5000 samples)
                p_secs = d_array[0][m:m+5000]
                p_f, p_t, p_Sxx = spectrogram(p_secs, fs=5000, return_onesided=False)
                p_SS = np.log1p(p_Sxx)
                arr = p_SS[:] / np.max(p_SS)
                all_X.append(arr)
                all_Y.append(i)
    
    return all_X, all_Y

In [6]:
# Load both Patient 1 and Patient 2 data
types = ['Patient_1_interictal_segment', 'Patient_1_preictal_segment']
all_X1, all_Y1 = load_patient_data(1, types, 18)

types = ['Patient_2_interictal_segment', 'Patient_2_preictal_segment']
all_X2, all_Y2 = load_patient_data(2, types, 18)

# Combine data from both patients
all_X = all_X1 + all_X2
all_Y = all_Y1 + all_Y2

In [7]:
# Shuffle the data
dataset = list(zip(all_X, all_Y))
random.shuffle(dataset)
all_X, all_Y = zip(*dataset)

In [8]:
# Convert to numpy arrays
x_train = np.array(all_X[:42000])
y_train = np.array(all_Y[:42000])
x_test = np.array(all_X[42000:])
y_test = np.array(all_Y[42000:])


In [9]:
# Reshape and normalize the data
img_rows, img_cols = 256, 22
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [10]:
# Convert class vectors to binary class matrices (one-hot encoding)
num_classes = 2
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)


# Lstms with attention mechanism

In [45]:
from tensorflow.keras.layers import LSTM, TimeDistributed

# Attention mechanism block (Self-attention)
def attention_block(inputs):
    channels = inputs.shape[-1]
    attention = GlobalAveragePooling2D()(inputs)
    attention = Dense(channels // 8, activation='relu')(attention)
    attention = Dense(channels, activation='sigmoid')(attention)
    attention = Reshape((1, 1, channels))(attention)
    attention = Multiply()([inputs, attention])
    return attention

# CNN-LSTM model with attention mechanism
input_shape = (img_rows, img_cols, 1)
inputs = Input(shape=input_shape)

# CNN layers for spatial feature extraction
x = Conv2D(32, kernel_size=(5, 5), activation='relu')(inputs)
x = Conv2D(32, kernel_size=(3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)

# Apply attention mechanism after convolutional layers
x = attention_block(x)

# Reshaping for LSTM input
# Reshape the data to (batch_size, time_steps, features) to fit LSTM requirements
x = TimeDistributed(Flatten())(x)  # Flatten across time

# LSTM layers for temporal feature extraction
x = LSTM(64, return_sequences=False)(x)  # Add LSTM layer (no return sequences)
x = Dropout(0.5)(x)

# Output layer
outputs = Dense(2, activation='sigmoid')(x)

# Build the model
model = Model(inputs, outputs)



NameError: name 'X_train_new' is not defined

# Using 'binary_crossentropy'

In [26]:
# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Print model summary
model.summary()

# Train the model
batch_size = 128
epochs = 20
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=1)



Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 256, 22,   │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 252, 18,   │        832 │ input_layer_3[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 250, 16,   │      9,248 │ conv2d_6[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 125, 8,    │          0 │ conv2d_7[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_6 (Dropout) │ (None, 125, 8,    │          0 │ max_pooling2d_3[… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 32)        │          0 │ dropout_6[0][0]   │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 4)         │        132 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 32)        │        160 │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_3 (Reshape) │ (None, 1, 1, 32)  │          0 │ dense_10[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_3          │ (None, 125, 8,    │          0 │ dropout_6[0][0],  │
│ (Multiply)          │ 32)               │            │ reshape_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_3  │ (None, 125, 256)  │          0 │ multiply_3[0][0]  │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ (None, 64)        │     82,176 │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_7 (Dropout) │ (None, 64)        │          0 │ lstm_3[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 2)         │        130 │ dropout_7[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 92,678 (362.02 KB)

 Trainable params: 92,678 (362.02 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20


2024-10-21 00:06:22.457875: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_9_1/dropout_6_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


296/296 ━━━━━━━━━━━━━━━━━━━━ 17s 40ms/step - accuracy: 0.5567 - loss: 0.6221 - val_accuracy: 0.5881 - val_loss: 0.5802
Epoch 2/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.6278 - loss: 0.5749 - val_accuracy: 0.7438 - val_loss: 0.4886
Epoch 3/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.7607 - loss: 0.4595 - val_accuracy: 0.8536 - val_loss: 0.3065
Epoch 4/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.8017 - loss: 0.3822 - val_accuracy: 0.6064 - val_loss: 0.5510
Epoch 5/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.6895 - loss: 0.5288 - val_accuracy: 0.8410 - val_loss: 0.3744
Epoch 6/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.8179 - loss: 0.3668 - val_accuracy: 0.8136 - val_loss: 0.4079
Epoch 7/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.8089 - loss: 0.3826 - val_accuracy: 0.8652 - val_loss: 0.3412
Epoch 8/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.8373 - loss: 0.3406 - val_accurac

In [27]:
# Evaluate the model on test data
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.13678957521915436
Test accuracy: 0.9458333253860474


# Using dice as loss function

In [28]:
from tensorflow.keras.layers import LSTM, TimeDistributed

# Attention mechanism block (Self-attention)
def attention_block(inputs):
    channels = inputs.shape[-1]
    attention = GlobalAveragePooling2D()(inputs)
    attention = Dense(channels // 8, activation='relu')(attention)
    attention = Dense(channels, activation='sigmoid')(attention)
    attention = Reshape((1, 1, channels))(attention)
    attention = Multiply()([inputs, attention])
    return attention

# CNN-LSTM model with attention mechanism
input_shape = (img_rows, img_cols, 1)
inputs = Input(shape=input_shape)

# CNN layers for spatial feature extraction
x = Conv2D(32, kernel_size=(5, 5), activation='relu')(inputs)
x = Conv2D(32, kernel_size=(3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)

# Apply attention mechanism after convolutional layers
x = attention_block(x)

# Reshaping for LSTM input
# Reshape the data to (batch_size, time_steps, features) to fit LSTM requirements
x = TimeDistributed(Flatten())(x)  # Flatten across time

# LSTM layers for temporal feature extraction
x = LSTM(64, return_sequences=False)(x)  # Add LSTM layer (no return sequences)
x = Dropout(0.5)(x)

# Output layer
outputs = Dense(2, activation='sigmoid')(x)

# Build the model
model = Model(inputs, outputs)



In [29]:
import tensorflow.keras.backend as K
def dice_loss(y_true, y_pred):
    intersection = K.sum(y_true * y_pred)
    return 1 - (2. * intersection + 1) / (K.sum(y_true) + K.sum(y_pred) + 1)
# Compile the model
model2 = Model(inputs, outputs)
model2.compile(loss=dice_loss, optimizer='adam', metrics=['accuracy'])


In [30]:
# Train the model
batch_size = 128
epochs = 20
model2.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=1)


Epoch 1/20


2024-10-21 00:10:02.680374: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_13_1/dropout_8_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


296/296 ━━━━━━━━━━━━━━━━━━━━ 16s 40ms/step - accuracy: 0.5223 - loss: 0.3579 - val_accuracy: 0.4848 - val_loss: 0.3325
Epoch 2/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.5145 - loss: 0.3308 - val_accuracy: 0.4848 - val_loss: 0.3160
Epoch 3/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.5112 - loss: 0.3168 - val_accuracy: 0.4848 - val_loss: 0.3154
Epoch 4/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.5145 - loss: 0.3164 - val_accuracy: 0.4848 - val_loss: 0.3155
Epoch 5/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.5110 - loss: 0.3160 - val_accuracy: 0.4848 - val_loss: 0.3155
Epoch 6/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.5186 - loss: 0.3154 - val_accuracy: 0.4848 - val_loss: 0.3153
Epoch 7/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.5299 - loss: 0.3153 - val_accuracy: 0.4848 - val_loss: 0.3156
Epoch 8/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 20s 36ms/step - accuracy: 0.5219 - loss: 0.3153 - val_accurac

In [31]:
# Evaluate the model on test data
score2 = model2.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score2[0])
print('Test accuracy:', score2[1])

Test loss: 0.31329789757728577
Test accuracy: 0.512499988079071


# Using MSE

In [32]:
from tensorflow.keras.layers import LSTM, TimeDistributed

# Attention mechanism block (Self-attention)
def attention_block(inputs):
    channels = inputs.shape[-1]
    attention = GlobalAveragePooling2D()(inputs)
    attention = Dense(channels // 8, activation='relu')(attention)
    attention = Dense(channels, activation='sigmoid')(attention)
    attention = Reshape((1, 1, channels))(attention)
    attention = Multiply()([inputs, attention])
    return attention

# CNN-LSTM model with attention mechanism
input_shape = (img_rows, img_cols, 1)
inputs = Input(shape=input_shape)

# CNN layers for spatial feature extraction
x = Conv2D(32, kernel_size=(5, 5), activation='relu')(inputs)
x = Conv2D(32, kernel_size=(3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)

# Apply attention mechanism after convolutional layers
x = attention_block(x)

# Reshaping for LSTM input
# Reshape the data to (batch_size, time_steps, features) to fit LSTM requirements
x = TimeDistributed(Flatten())(x)  # Flatten across time

# LSTM layers for temporal feature extraction
x = LSTM(64, return_sequences=False)(x)  # Add LSTM layer (no return sequences)
x = Dropout(0.5)(x)

# Output layer
outputs = Dense(2, activation='sigmoid')(x)

# Build the model
model3 = Model(inputs, outputs)



In [33]:
model3.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])


In [34]:
# Train the model
batch_size = 128
epochs = 20
model3.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=1)


Epoch 1/20


2024-10-21 00:13:56.059794: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_15_1/dropout_10_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


296/296 ━━━━━━━━━━━━━━━━━━━━ 16s 39ms/step - accuracy: 0.5364 - loss: 0.2432 - val_accuracy: 0.5752 - val_loss: 0.2088
Epoch 2/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.6043 - loss: 0.2094 - val_accuracy: 0.6764 - val_loss: 0.2045
Epoch 3/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.6531 - loss: 0.2025 - val_accuracy: 0.7579 - val_loss: 0.1519
Epoch 4/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.7745 - loss: 0.1491 - val_accuracy: 0.8467 - val_loss: 0.1164
Epoch 5/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.8259 - loss: 0.1226 - val_accuracy: 0.8714 - val_loss: 0.0886
Epoch 6/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.8714 - loss: 0.0927 - val_accuracy: 0.9105 - val_loss: 0.0678
Epoch 7/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.8916 - loss: 0.0805 - val_accuracy: 0.9198 - val_loss: 0.0648
Epoch 8/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.8983 - loss: 0.0769 - val_accurac

In [35]:
# Evaluate the model on test data
score3 = model3.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score3[0])
print('Test accuracy:', score3[1])

Test loss: 0.049094054847955704
Test accuracy: 0.940833330154419


In [36]:
print(score3)

[0.049094054847955704, 0.940833330154419]


In [40]:
# Create a DataFrame
data = {
    'Model': ['Binary Cross-Entropy', 
              'Dice', 
              'MSE'],
    'Loss': [score[0], score2[0], score3[0]],
    'Accuracy': [score[1], score2[1], score3[1]]
}

results_df = pd.DataFrame(data)

# Display the DataFrame
results_df

,Model,Loss,Accuracy
0,Binary Cross-Entropy,0.136790,0.945833
1,Dice,0.313298,0.512500
2,MSE,0.049094,0.940833
